# Incoporating comments from [PR](https://github.com/cal-itp/data-analyses/pull/1266)
<i>tidier way, and now there's no need to define and redefine your merge columns: (1) merge schedule with rt_vs_schedule. merge columns are schedule_gtfs_dataset_key and date, (2) merge in crosswalk that fills in organization info + NTD info. merge columns are schedule_gtfs_dataset_key and date.
addressing this comment would need to be done in rt_scheduled_v_ran/scripts/rt_v_scheduled_operator.py...don't bring in organization stuff here! At this level, when things are created, it's all done using schedule_gtfs_dataset_key, and only when it's brought together in the next downstream step in gtfs_digest, crosswalk is merged in once for all.</i>

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from segment_speed_utils import (
    gtfs_schedule_wrangling,
    helpers,
    metrics,
    time_series_utils,
)
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
rt_dates.y2024_dates + rt_dates.y2023_dates

['2024-01-17',
 '2024-02-14',
 '2024-03-13',
 '2024-04-17',
 '2024-05-22',
 '2024-06-12',
 '2024-07-17',
 '2024-08-14',
 '2024-09-18',
 '2024-10-16',
 '2023-03-15',
 '2023-04-12',
 '2023-05-17',
 '2023-06-14',
 '2023-07-12',
 '2023-08-15',
 '2023-09-13',
 '2023-10-11',
 '2023-11-15',
 '2023-12-13']

In [4]:
dict_inputs = GTFS_DATA_DICT.rt_vs_schedule_tables

In [5]:
RT_SCHED_GCS

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/'

In [6]:
analysis_date = "2024-05-22"

In [7]:
TRIP_EXPORT = dict_inputs.vp_trip_metrics

In [8]:
trip_df = pd.read_parquet(f"{RT_SCHED_GCS}{TRIP_EXPORT}_{analysis_date}.parquet")

In [9]:
trip_df.head(1)

,schedule_gtfs_dataset_key,trip_instance_key,route_id,direction_id,scheduled_service_minutes,total_vp,rt_service_minutes,minutes_atleast1_vp,minutes_atleast2_vp,vp_in_shape,sched_rt_category,time_of_day,peak_offpeak,vp_per_minute,pct_in_shape,pct_rt_journey_atleast1_vp,pct_rt_journey_atleast2_vp,pct_sched_journey_atleast1_vp,pct_sched_journey_atleast2_vp,rt_sched_journey_difference,is_early,is_ontime,is_late
0,d9d0325e50e50064e3cc8384b1751d67,000054ea29bb07adcfa0fc0f13c4a030,29,1.00,64.00,152,75.98,76,71,152,schedule_and_vp,Midday,offpeak,2.00,1.00,1.00,0.93,1.00,1.00,11.98,0,0,1


In [10]:
OP_EXPORT = dict_inputs.vp_operator_metrics

In [11]:
OP_EXPORT

'vp_operator/operator_metrics'

In [12]:
url = "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/vp_operator/operator_metrics_2024-05-22.parquet"

In [13]:
may_rt_metrics = pd.read_parquet(url)

In [14]:
SCHED_GCS

'gs://calitp-analytics-data/data-analyses/gtfs_schedule/'

In [15]:
GTFS_DATA_DICT.schedule_tables.operator_scheduled_stats

'operator_profiles/operator_scheduled_stats'

In [16]:
may_op_scheduled = gpd.read_parquet("gs://calitp-analytics-data/data-analyses/gtfs_schedule/operator_profiles/operator_routes_2024-05-22.parquet")

In [17]:
may_rt_metrics.head(1)

,schedule_gtfs_dataset_key,vp_per_min_agency,spatial_accuracy_agency
0,0139b1253130b33adcd4b3a4490530d2,2.45,63.42


In [18]:
may_op_scheduled.columns

Index(['shape_array_key', 'geometry', 'feed_key', 'schedule_gtfs_dataset_key',
       'route_id', 'direction_id', 'route_key', 'route_length',
       'route_length_miles', 'is_downtown_local', 'is_local', 'is_coverage',
       'is_rapid', 'is_express', 'is_rail', 'name',
       'organization_source_record_id', 'organization_name'],
      dtype='object')